Let's first create a simple receiver using the Spead2 library. We'll need to import `spead2` and `spead2.recv`.


In [ ]:
import spead2
import spead2.recv

A `ThreadPool` is what the Spead2 library uses to manage C threads, which we can attach streams to in order to send and receive data. Streams require a `ThreadPool` argument when constructed, which means we need one even if we only plan on using one stream.

In [ ]:
thread_pool = spead2.ThreadPool()

In a larger scale application, we could at this point instantiate a `spead2.MemoryPool` . This would allow us to block off a section of memory for incoming data to be quickly stored to. We will not use them in this small example, but you can read more about them at this link: https://spead2.readthedocs.io/en/latest/py-recv.html#spead2.MemoryPool


We now create a receiving `spead2.recv.Stream` linked to our ThreadPool . This abstraction will handle the capture of all Spead packets and allow us to easily convert it to a useful form.


In [ ]:
stream = spead2.recv.Stream(thread_pool)

We listen on port 8888 for any udp traffic.


In [ ]:
stream.add_udp_reader(8888)

Our stream will now begin to collect data in the form of Spead2 Heaps, which are defined in the protocol. Spead2 provides us with a class, `spead2.ItemGroup`, which handles the parsing of the heaps into a more workable form, namely `spead2.Item`s. An `ItemGroup` will accumulate a set of `Item`s which correspond to the data that was sent to us.


In [ ]:
item_group = spead2.ItemGroup()

The `Stream.next()` method will wait until it reconstructs an entire heap before returning it, at which point we can use the `ItemGroup.update` method to begin processing the heap into its constituent items. The `Stream.next()` method will block program execution until such a heap arrives, and will raise `spead2.Stopped` if it encounters an end-of-heap marker.


In [ ]:
for received_heap in stream:
    item_group.update(stream)

At this point, we can close the stream.

In [ ]:
stream.stop()

We can get a dictionary of `Item`s from our `ItemGroup` with its `ItemGroup.items` method. Let's try printing their names and values.


In [ ]:
for (key, item) in item_group.items():
    print(item.name, ":", item.value)

Those are the basics of receiving data in Spead2. To dive deeper into receiving, you can read through the documentation at https://spead2.readthedocs.io/en/latest/py-recv.html
